## Set project path

In [ ]:
#Keep the 360 video file wihin the project path
PROJECT_PATH = "/home/aritra/360_point_cloud/fengyu_220114_1f"
video_file_name = "VID_20220114_160120_00_035.mp4"

## Extract frames from the video

In [ ]:
import sys
import argparse
import cv2
import os
print(cv2.__version__)

def extractImages(pathIn, pathOut):
    vidcap = cv2.VideoCapture(pathIn)
    success,image = vidcap.read()
    count = 0
    success = True
    while success:
      vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))    # for 1 frame in 1 sec.
      #vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000*4))   # for 1 frame in 4 sec
      success,image = vidcap.read()
      print ('Read a new frame: ', success)
      cv2.imwrite( pathOut + "/%#05d.jpg" % count, image)     # save frame as JPEG file
      count += 1 # change counter for increase or decrease the number of frame


input_path = os.path.join(PROJECT_PATH,video_file_name)
!mkdir $PROJECT_PATH/images
output_path = os.path.join(PROJECT_PATH,"images")
extractImages(input_path, output_path)

## Extract Metadata

In [ ]:
! /home/aritra/OpenSfM/bin/opensfm extract_metadata $PROJECT_PATH

## Modify "camera_models.json" file
For equirectangular images

In [ ]:
import json
import os
f = open(os.path.join(PROJECT_PATH, 'camera_models.json'))
data = json.load(f)
#data["all"] = data.pop(list(data.keys())[0])
key = str(list(data.keys())[0])
print(key)
data[key]["projection_type"] = "equirectangular"
del data[key]["focal"]
del data[key]["k1"]
del data[key]["k2"]
print(data)
with open(os.path.join(PROJECT_PATH, 'camera_models.json'), "w") as outfile:
    json.dump(data, outfile,indent =2)

## Run OpenSfM commands for creating point clouds

In [ ]:
! /home/aritra/OpenSfM/bin/opensfm detect_features $PROJECT_PATH
! /home/aritra/OpenSfM/bin/opensfm match_features $PROJECT_PATH
! /home/aritra/OpenSfM/bin/opensfm create_tracks $PROJECT_PATH
! /home/aritra/OpenSfM/bin/opensfm reconstruct $PROJECT_PATH
! /home/aritra/OpenSfM/bin/opensfm bundle $PROJECT_PATH
! /home/aritra/OpenSfM/bin/opensfm undistort $PROJECT_PATH
! /home/aritra/OpenSfM/bin/opensfm compute_depthmaps $PROJECT_PATH

## Extract Metadata and Detect Features for undistorted perspective images

In [ ]:
! /home/aritra/OpenSfM/bin/opensfm extract_metadata $PROJECT_PATH/undistorted
! /home/aritra/OpenSfM/bin/opensfm detect_features $PROJECT_PATH/undistorted

## Create a "camera_models.json" file for perspective images

In [ ]:
f = open(os.path.join(PROJECT_PATH , "undistorted", 'camera_models.json'))
data = json.load(f)
data1 = data.copy()
data1["perspective_panorama_camera"] = data1.pop(list(data1.keys())[0])

data2 = {**data1, **data}
print(data2)

with open(os.path.join(PROJECT_PATH, "undistorted", 'camera_models.json'), "w") as outfile:
    json.dump(data2, outfile, indent =2)

## Export the sparse point cloud into colmap format

In [ ]:
! /home/aritra/OpenSfM/bin/opensfm export_colmap $PROJECT_PATH/undistorted

## Create dense folder for colmap reconstruction & Undistort images

In [ ]:
!mkdir $PROJECT_PATH/undistorted/colmap_dense
!colmap image_undistorter\
    --image_path $PROJECT_PATH/undistorted/images\
    --input_path $PROJECT_PATH/undistorted/colmap_export\
    --output_path $PROJECT_PATH/undistorted/colmap_dense

## Colmap stereo matching

In [ ]:
!colmap patch_match_stereo\
    --workspace_path $PROJECT_PATH/undistorted/colmap_dense\
    --workspace_format COLMAP \
    --PatchMatchStereo.max_image_size 3840 \
    --PatchMatchStereo.window_radius 9 \
    --PatchMatchStereo.geom_consistency true \
    --PatchMatchStereo.filter_min_ncc 0.07\
    --PatchMatchStereo.depth_min 0.0 \
    --PatchMatchStereo.depth_max 20.0

## Colmap stereo fusion

In [ ]:
!colmap stereo_fusion\
    --workspace_path $PROJECT_PATH/undistorted/colmap_dense\
    --output_path $PROJECT_PATH/undistorted/colmap_dense/fused.ply

## Merge Colmap point cloud and mvs point cloud for denser one
At present this doesnot work for creating a colored point cloud. However, It can output a denser point cloud

In [ ]:
import open3d as o3d
import numpy as np
pcd1 = o3d.io.read_point_cloud(os.path.join(PROJECT_PATH,"/undistorted/depthmaps/merged.ply"))
p1_load = np.asarray(pcd1.points)
pcd2 = o3d.io.read_point_cloud(os.path.join(PROJECT_PATH,"/undistorted/colmap_dense/fused.ply"))
p2_load = np.asarray(pcd2.points)
#print(p2_load)
p3_load = np.concatenate((p1_load,p2_load), axis=0)
p1_color = np.asarray(pcd1.colors)
p2_color = np.asarray(pcd2.colors)
p3_color = np.concatenate((p1_color,p2_color), axis=0)
print(p3_color)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(p3_load)
pcd.colors = o3d.utility.Vector3dVector(p3_color)
#print(np.asarray(pcd.points))
#print(np.asarray(pcd.colors))
o3d.io.write_point_cloud(os.path.join(PROJECT_PATH,"dense.ply"), pcd)